In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import layers

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [3]:
import numpy as np

x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

In [4]:
model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 0s 714us/step - loss: 11.9091
Epoch 2/10
8/8 [==============================] - 0s 714us/step - loss: 12.2097
Epoch 3/10
8/8 [==============================] - 0s 714us/step - loss: 12.9565
Epoch 4/10
8/8 [==============================] - 0s 714us/step - loss: 14.2079
Epoch 5/10
8/8 [==============================] - 0s 571us/step - loss: 16.1753
Epoch 6/10
8/8 [==============================] - 0s 714us/step - loss: 18.9806
Epoch 7/10
8/8 [==============================] - 0s 714us/step - loss: 22.4110
Epoch 8/10
8/8 [==============================] - 0s 715us/step - loss: 26.2437
Epoch 9/10
8/8 [==============================] - 0s 714us/step - loss: 29.9652
Epoch 10/10
8/8 [==============================] - 0s 714us/step - loss: 34.1463


In [5]:
model.evaluate(x_train, y_train)

32/32 [==============================] - 0s 452us/step - loss: 36.4869


36.48687744140625

Multi-input model

In [6]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [7]:
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answers = to_categorical(answers)

In [8]:
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 2s 56ms/step - loss: 6.2144 - acc: 0.0030
Epoch 2/10
8/8 [==============================] - 0s 57ms/step - loss: 6.1963 - acc: 0.0440
Epoch 3/10
8/8 [==============================] - 0s 51ms/step - loss: 6.1464 - acc: 0.0190
Epoch 4/10
8/8 [==============================] - 0s 45ms/step - loss: 6.0449 - acc: 0.0080
Epoch 5/10
8/8 [==============================] - 0s 49ms/step - loss: 5.9787 - acc: 0.0110
Epoch 6/10
8/8 [==============================] - 0s 48ms/step - loss: 5.9065 - acc: 0.0110
Epoch 7/10
8/8 [==============================] - 0s 47ms/step - loss: 5.8033 - acc: 0.0130
Epoch 8/10
8/8 [==============================] - 0s 50ms/step - loss: 5.7136 - acc: 0.0220
Epoch 9/10
8/8 [==============================] - 0s 46ms/step - loss: 5.6337 - acc: 0.0280
Epoch 10/10
8/8 [==============================] - 0s 48ms/step - loss: 5.5515 - acc: 0.0350


In [9]:
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 1s 49ms/step - loss: 5.4784 - acc: 0.0390
Epoch 2/10
8/8 [==============================] - 0s 50ms/step - loss: 5.4269 - acc: 0.0450
Epoch 3/10
8/8 [==============================] - 0s 54ms/step - loss: 5.3455 - acc: 0.0640: 0s - loss: 5.3862 - acc: 
Epoch 4/10
8/8 [==============================] - 0s 49ms/step - loss: 5.2750 - acc: 0.0720
Epoch 5/10
8/8 [==============================] - 0s 51ms/step - loss: 5.2035 - acc: 0.0870
Epoch 6/10
8/8 [==============================] - 0s 49ms/step - loss: 5.1741 - acc: 0.0900
Epoch 7/10
8/8 [==============================] - 0s 51ms/step - loss: 5.0887 - acc: 0.0950
Epoch 8/10
8/8 [==============================] - 0s 50ms/step - loss: 5.0172 - acc: 0.1140
Epoch 9/10
8/8 [==============================] - 0s 48ms/step - loss: 4.9948 - acc: 0.1210
Epoch 10/10
8/8 [==============================] - 0s 48ms/step - loss: 4.9158 - acc: 0.1300


Multi-output model

In [10]:
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, 
                                 activation='softmax', 
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, 
              [age_prediction, income_prediction, gender_prediction])

In [11]:
model.compile(optimizer='rmsprop', 
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop', 
              loss={'age': 'mse', 
                    'income': 'categorical_crossentropy', 
                    'gender': 'binary_crossentropy'})

In [12]:
model.compile(optimizer='rmsprop', 
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], 
              loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop', 
              loss={'age': 'mse', 
                    'income': 'categorical_crossentropy', 
                    'gender': 'binary_crossentropy'}, 
              loss_weights={'age': 0.25, 
                            'income': 1., 
                            'gender': 10.})